In [27]:
import pandas as pd
from datetime import datetime
import ta
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [28]:
data = pd.read_csv("niftybank.csv")
data.head()

,date,open,high,low,close,volume
0,2017-07-17 09:14:00,24011.40,24011.40,24011.40,24011.40,0
1,2017-07-17 09:15:00,24011.40,24014.40,23974.95,23974.95,0
2,2017-07-17 09:16:00,23975.45,23981.55,23963.15,23978.95,0
3,2017-07-17 09:17:00,23977.25,23995.30,23976.00,23984.60,0
4,2017-07-17 09:18:00,23983.25,23989.95,23983.25,23987.10,0


In [29]:
data["Date"] = data["date"].str.split(" ").str[0]
data["Time"] = data["date"].str.split(" ").str[1].str.split("+").str[0]
data["Date_Time"] = data['Date'] + " " + data["Time"]
data.drop("Date", axis = 1, inplace = True)
data



,date,open,high,low,close,volume,Time,Date_Time
0,2017-07-17 09:14:00,24011.40,24011.40,24011.40,24011.40,0,09:14:00,2017-07-17 09:14:00
1,2017-07-17 09:15:00,24011.40,24014.40,23974.95,23974.95,0,09:15:00,2017-07-17 09:15:00
2,2017-07-17 09:16:00,23975.45,23981.55,23963.15,23978.95,0,09:16:00,2017-07-17 09:16:00
3,2017-07-17 09:17:00,23977.25,23995.30,23976.00,23984.60,0,09:17:00,2017-07-17 09:17:00
4,2017-07-17 09:18:00,23983.25,23989.95,23983.25,23987.10,0,09:18:00,2017-07-17 09:18:00
...,...,...,...,...,...,...,...,...
616014,2024-03-07 15:25:00,47815.65,47815.65,47788.80,47802.45,0,15:25:00,2024-03-07 15:25:00
616015,2024-03-07 15:26:00,47805.65,47805.65,47790.45,47795.80,0,15:26:00,2024-03-07 15:26:00
616016,2024-03-07 15:27:00,47797.10,47802.95,47782.00,47800.35,0,15:27:00,2024-03-07 15:27:00
616017,2024-03-07 15:28:00,47802.70,47806.05,47783.75,47785.70,0,15:28:00,2024-03-07 15:28:00


In [30]:
data["Date_Time"] = pd.to_datetime(data["Date_Time"], format = "%Y-%m-%d %H:%M:%S")
data.index = data['Date_Time']
data.drop(["date"], axis = 1, inplace = True)
data["Next Date_Time"] = data["Date_Time"].shift(-1)
data

,open,high,low,close,volume,Time,Date_Time,Next Date_Time
Date_Time,,,,,,,,
2017-07-17 09:14:00,24011.40,24011.40,24011.40,24011.40,0,09:14:00,2017-07-17 09:14:00,2017-07-17 09:15:00
2017-07-17 09:15:00,24011.40,24014.40,23974.95,23974.95,0,09:15:00,2017-07-17 09:15:00,2017-07-17 09:16:00
2017-07-17 09:16:00,23975.45,23981.55,23963.15,23978.95,0,09:16:00,2017-07-17 09:16:00,2017-07-17 09:17:00
2017-07-17 09:17:00,23977.25,23995.30,23976.00,23984.60,0,09:17:00,2017-07-17 09:17:00,2017-07-17 09:18:00
2017-07-17 09:18:00,23983.25,23989.95,23983.25,23987.10,0,09:18:00,2017-07-17 09:18:00,2017-07-17 09:19:00
...,...,...,...,...,...,...,...,...
2024-03-07 15:25:00,47815.65,47815.65,47788.80,47802.45,0,15:25:00,2024-03-07 15:25:00,2024-03-07 15:26:00
2024-03-07 15:26:00,47805.65,47805.65,47790.45,47795.80,0,15:26:00,2024-03-07 15:26:00,2024-03-07 15:27:00
2024-03-07 15:27:00,47797.10,47802.95,47782.00,47800.35,0,15:27:00,2024-03-07 15:27:00,2024-03-07 15:28:00


In [31]:
def convert_timeframe(data, tf):
    data = data.copy()  # To avoid modifying the original DataFrame
    
    new_data = {
        "Date_Time": [],
        "Open": [],
        "High": [],
        "Low": [],
        "Close": []
    }
    
    step = 0
    for i in data.index:
        step += 1
        if step == 1:
            o = data.loc[i, "open"]
            t = i
            h = data.loc[i, "high"]
            l = data.loc[i, "low"]
        else:
            if data.loc[i, "high"] > h:
                h = data.loc[i, "high"]
            if data.loc[i, "low"] < l:
                l = data.loc[i, "low"]
            if step == tf or i.day != data.loc[i, "Next_Date_Time"].day:
                step = 0
                c = data.loc[i, "close"]
                new_data["Date_Time"].append(t)
                new_data["Open"].append(o)
                new_data["High"].append(h)
                new_data["Low"].append(l)
                new_data["Close"].append(c)
    
    return pd.DataFrame(new_data)


In [32]:
import pandas as pd

def convert_timeframe(data, tf):
    data = data.copy()  # To avoid modifying the original DataFrame
    
    new_data = {
        "Date_Time": [],
        "Open": [],
        "High": [],
        "Low": [],
        "Close": []
    }
    
    step = 0
    for i in range(len(data)):
        step += 1
        if step == 1:
            o = data.iloc[i]["open"]
            t = data.index[i]
            h = data.iloc[i]["high"]
            l = data.iloc[i]["low"]
        else:
            if data.iloc[i]["high"] > h:
                h = data.iloc[i]["high"]
            if data.iloc[i]["low"] < l:
                l = data.iloc[i]["low"]
            if step == tf or (i+1) == len(data) or data.index[i+1].day != data.index[i].day:
                step = 0
                c = data.iloc[i]["close"]
                new_data["Date_Time"].append(t)
                new_data["Open"].append(o)
                new_data["High"].append(h)
                new_data["Low"].append(l)
                new_data["Close"].append(c)
                if (i+1) < len(data):
                    t = data.index[i+1]
                    o = data.iloc[i+1]["open"]
                    h = data.iloc[i+1]["high"]
                    l = data.iloc[i+1]["low"]
    
    return pd.DataFrame(new_data)


In [33]:
data45 = convert_timeframe(data, 5)
data45_original = data45.copy()

In [34]:
data45.head()

,Date_Time,Open,High,Low,Close
0,2017-07-17 09:14:00,24011.40,24014.40,23963.15,23987.1
1,2017-07-17 09:19:00,23986.60,23998.25,23954.05,23982.9
2,2017-07-17 09:24:00,23983.55,23995.20,23967.95,23984.7
3,2017-07-17 09:29:00,23983.00,23984.85,23952.20,23954.9
4,2017-07-17 09:34:00,23954.15,23963.40,23921.20,23940.7


In [35]:
data45.to_excel("5EMA_BUY5_BN.xlsx")

In [36]:
data45.to_csv("5EMA_5BN.csv")

5EMA on 45 min Short Positions only

In [9]:
status = None
alert_candle = None
position = None
short_at = None
stop_loss = None

data45 = data45_original.copy()
data45["5EMA"] = ta.trend.EMAIndicator(data45["Close"], 5).ema_indicator()
day = None

for i in data45.index:
 
 if position == None:

    if alert_candle == None: # Previous Candle is not an alert candle

        if data45.loc[i, "5EMA"] < data45.loc[i, "Low"]: # Alert Candle
            alert_candle = i
            data45.loc[i, "ALERT"] = "ALERT"
    else: # Previous Candle is an alert candle -> Try to take postion


        if data45.loc[i, "Low"] < data45.loc[alert_candle, "Low"]:
            position = "SHORT"
            data45.loc[i, "POSITION"] = "ENTRY"

            if data45.loc[i, "Date_Time"].day != day:
                data45.loc[i, "Days fund required"] = 1
                day = data45.loc[i, "Date_Time"].day
            short_at = data45.loc[alert_candle, "Low"]

            stop_loss = data45.loc[alert_candle, "High"]
            original_sl = stop_loss
            target = short_at - 3*(stop_loss - short_at)

            data45.loc[i, "TARGET"] = target
            data45.loc[i, "SL"] = stop_loss
            data45.loc[i, "ENTRY_AT"] = short_at
            alert_candle = None

        elif data45.loc[i, "5EMA"] < data45.loc[i, "Low"]: # Check for once again aler
            alert_candle = i
            data45.loc[i, "ALERT"] = "ALERT"
        else:
            alert_candle = None

 else: # I already have a short position
    data45.loc[i, "POSITION"] = "HOLD"
    data45.loc[i, "SL"] = stop_loss
    data45.loc[i, "TARGET"] = target
    data45.loc[i, "ENTRY_AT"] = short_at

    if data45.loc[i, "Low"] <= target:
        data45.loc[i, "POSITION"] = "TRAIL"

        temp = target - (original_sl - short_at)
        stop_loss = target
        target = temp
        data45.loc[i, "SL"] = stop_loss
        data45.loc[i, "TARGET"] = target

    elif data45.loc[i, "High"] >= stop_loss:
            data45.loc[i, "POSITION"] = "EXIT"
            position = None
            profit = short_at - stop_loss
            data45.loc[i, "PROFIT"] = profit
            data45.loc[i, "EXIT_AT"] = stop_loss


In [10]:
data45.iloc[4222:4280] # Lets check any two random interval

,Date_Time,Open,High,Low,Close,5EMA,ALERT,POSITION,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
4222,2019-06-12 09:15:00,31219.30,31222.30,31074.00,31092.90,31206.800939,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4223,2019-06-12 10:00:00,31086.50,31107.35,30984.45,30992.50,31135.367293,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4224,2019-06-12 10:45:00,30991.10,31024.65,30954.10,30971.80,31080.844862,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4225,2019-06-12 11:30:00,30972.75,31006.00,30947.65,30979.40,31047.029908,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4226,2019-06-12 12:15:00,30979.05,31014.50,30943.20,30988.25,31027.436605,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4227,2019-06-12 13:00:00,30989.00,31030.80,30911.70,30946.05,31000.307737,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4228,2019-06-12 13:45:00,30945.15,31022.85,30936.45,30972.90,30991.171825,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4229,2019-06-12 14:30:00,30971.75,30979.40,30911.45,30960.75,30981.031216,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4230,2019-06-12 15:15:00,30957.70,31017.80,30949.45,31016.55,30992.870811,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4231,2019-06-13 09:15:00,30876.75,30954.80,30744.35,30745.45,30910.397207,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data45.describe()

,Date_Time,Open,High,Low,Close,5EMA,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
count,14784,14784.000000,14784.000000,14784.000000,14784.000000,14780.000000,840.0,5884.000000,5884.000000,5884.000000,983.000000,983.000000
mean,2020-11-12 07:44:36.663960832,32651.428382,32725.469981,32570.578774,32649.470285,32648.587105,1.0,32661.200918,33185.296117,33148.135240,63.903967,32653.239878
min,2017-07-17 09:14:00,16506.800000,16674.750000,16116.250000,16499.600000,16850.558633,1.0,16357.450000,17137.050000,17525.950000,-671.800000,17137.050000
25%,2019-03-13 12:48:45,26262.900000,26316.012500,26208.687500,26265.162500,26270.806495,1.0,26253.000000,26575.250000,26589.600000,-109.025000,26245.650000
50%,2020-11-11 10:22:30,31400.275000,31458.325000,31334.000000,31393.550000,31380.240673,1.0,31883.900000,32221.150000,32258.300000,-67.250000,31426.950000
75%,2022-07-12 09:26:15,38570.187500,38666.100000,38468.412500,38571.450000,38576.653683,1.0,38550.700000,39131.837500,39085.250000,204.950000,38873.950000
max,2024-03-07 15:15:00,48547.400000,48631.000000,48508.500000,48548.550000,48508.280192,1.0,48027.300000,48631.000000,48408.750000,3118.400000,48244.050000
std,NaN,7590.527125,7594.645637,7586.580701,7590.748730,7587.438620,0.0,7623.805799,7653.552673,7634.994598,341.304420,7581.398910


In [12]:
data45.index = data45["Date_Time"]
data45["PROFIT"].resample("A").sum()


Date_Time
2017-12-31      496.15
2018-12-31     2161.25
2019-12-31     8037.00
2020-12-31    19188.35
2021-12-31     6993.30
2022-12-31    18934.55
2023-12-31     5698.85
2024-12-31     1308.15
Freq: A-DEC, Name: PROFIT, dtype: float64

In [13]:
data45

,Date_Time,Open,High,Low,Close,5EMA,ALERT,POSITION,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
Date_Time,,,,,,,,,,,,,,
2017-07-17 09:14:00,2017-07-17 09:14:00,24011.40,24014.40,23914.40,23977.30,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-17 09:59:00,2017-07-17 09:59:00,23976.60,24003.20,23952.05,23988.85,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-17 10:44:00,2017-07-17 10:44:00,23988.95,24021.80,23967.75,23979.65,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-17 11:29:00,2017-07-17 11:29:00,23980.35,23990.60,23964.90,23976.00,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-17 12:14:00,2017-07-17 12:14:00,23975.15,23997.40,23963.75,23988.55,23982.250000,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-07 12:15:00,2024-03-07 12:15:00,47837.45,47860.70,47777.25,47805.40,47852.107197,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-07 13:00:00,2024-03-07 13:00:00,47802.90,47883.60,47792.10,47854.45,47852.888131,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-07 13:45:00,2024-03-07 13:45:00,47856.60,47939.80,47830.40,47931.65,47879.142087,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


5EMA 45min Buy Only

In [14]:
data45B = convert_timeframe(data, 45)

In [15]:
data45B_original = data45B.copy()
data45B

,Date_Time,Open,High,Low,Close
0,2017-07-17 09:14:00,24011.40,24014.40,23914.40,23977.30
1,2017-07-17 09:59:00,23976.60,24003.20,23952.05,23988.85
2,2017-07-17 10:44:00,23988.95,24021.80,23967.75,23979.65
3,2017-07-17 11:29:00,23980.35,23990.60,23964.90,23976.00
4,2017-07-17 12:14:00,23975.15,23997.40,23963.75,23988.55
...,...,...,...,...,...
14779,2024-03-07 12:15:00,47837.45,47860.70,47777.25,47805.40
14780,2024-03-07 13:00:00,47802.90,47883.60,47792.10,47854.45
14781,2024-03-07 13:45:00,47856.60,47939.80,47830.40,47931.65
14782,2024-03-07 14:30:00,47935.65,47975.60,47803.95,47857.00


In [16]:
status = None
alert_candle = None
position = None
long_at = None
stop_loss = None

data45B = data45B_original.copy()
data45B["5EMA"] = ta.trend.EMAIndicator(data45B["Close"], 5).ema_indicator()
day = None

for i in data45B.index:
 
 if position == None:

    if alert_candle == None: # Previous Candle is not an alert candle

        if data45B.loc[i, "5EMA"] > data45B.loc[i, "High"]: # Alert Candle
            alert_candle = i
            data45B.loc[i, "ALERT"] = "ALERT"

    else: # Previous Candle is an alert candle -> Try to take postion


        if data45B.loc[i, "High"] > data45B.loc[alert_candle, "High"]:
            position = "LONG"
            data45B.loc[i, "POSITION"] = "ENTRY"

            if data45B.loc[i, "Date_Time"].day != day:
                data45B.loc[i, "Days fund required"] = 1
                day = data45B.loc[i, "Date_Time"].day

            long_at = data45B.loc[alert_candle, "High"]

            stop_loss = data45B.loc[alert_candle, "Low"]
            original_sl = stop_loss
            target = long_at + 3*(long_at - stop_loss)

            data45B.loc[i, "TARGET"] = target
            data45B.loc[i, "SL"] = stop_loss
            data45B.loc[i, "ENTRY_AT"] = long_at
            alert_candle = None

        elif data45B.loc[i, "5EMA"] > data45B.loc[i, "High"]: # Check for once again alert
            alert_candle = i
            data45B.loc[i, "ALERT"] = "ALERT"
        else:
            alert_candle = None

 else: # I already have a long position
    data45B.loc[i, "POSITION"] = "HOLD"
    data45B.loc[i, "SL"] = stop_loss
    data45B.loc[i, "TARGET"] = target
    data45B.loc[i, "ENTRY_AT"] = long_at

    if data45B.loc[i, "High"] >= target:
        data45B.loc[i, "POSITION"] = "TRAIL"

        temp = target + (long_at - original_sl)
        stop_loss = target
        target = temp
        data45B.loc[i, "SL"] = stop_loss
        data45B.loc[i, "TARGET"] = target

    elif data45B.loc[i, "Low"] <= stop_loss:
            data45B.loc[i, "POSITION"] = "EXIT"
            position = None
            profit = stop_loss - long_at
            data45B.loc[i, "PROFIT"] = profit
            data45B.loc[i, "EXIT_AT"] = stop_loss


In [17]:
data45B.loc[4353:4373] # Lets check any two random interval

,Date_Time,Open,High,Low,Close,5EMA,ALERT,POSITION,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
4353,2019-07-02 13:00:00,31192.35,31280.10,31186.15,31242.60,31223.471965,nan,HOLD,NaN,31471.0,31060.6,31163.2,NaN,NaN
4354,2019-07-02 13:45:00,31243.70,31329.30,31231.35,31303.20,31250.047977,nan,HOLD,NaN,31471.0,31060.6,31163.2,NaN,NaN
4355,2019-07-02 14:30:00,31303.00,31321.20,31238.70,31287.65,31262.581984,nan,HOLD,NaN,31471.0,31060.6,31163.2,NaN,NaN
4356,2019-07-02 15:15:00,31285.55,31298.05,31260.85,31268.75,31264.637990,nan,HOLD,NaN,31471.0,31060.6,31163.2,NaN,NaN
4357,2019-07-03 09:15:00,31347.70,31397.90,31238.50,31297.05,31275.441993,nan,HOLD,NaN,31471.0,31060.6,31163.2,NaN,NaN
4358,2019-07-03 10:00:00,31296.95,31371.60,31286.95,31349.90,31300.261329,nan,HOLD,NaN,31471.0,31060.6,31163.2,NaN,NaN
4359,2019-07-03 10:45:00,31349.65,31357.45,31317.05,31345.40,31315.307552,nan,HOLD,NaN,31471.0,31060.6,31163.2,NaN,NaN
4360,2019-07-03 11:30:00,31345.30,31368.75,31293.40,31361.50,31330.705035,nan,HOLD,NaN,31471.0,31060.6,31163.2,NaN,NaN
4361,2019-07-03 12:15:00,31360.35,31449.90,31360.35,31423.50,31361.636690,nan,HOLD,NaN,31471.0,31060.6,31163.2,NaN,NaN
4362,2019-07-03 13:00:00,31421.60,31467.15,31385.80,31448.25,31390.507793,nan,HOLD,NaN,31471.0,31060.6,31163.2,NaN,NaN


In [18]:
data45B.describe()

,Date_Time,Open,High,Low,Close,5EMA,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
count,14784,14784.000000,14784.000000,14784.000000,14784.000000,14780.000000,678.0,6321.000000,6321.000000,6321.000000,810.000000,810.000000
mean,2020-11-12 07:44:36.663960832,32651.428382,32725.469981,32570.578774,32649.470285,32648.587105,1.0,32072.090864,31569.239480,31603.939298,79.298827,32723.985062
min,2017-07-17 09:14:00,16506.800000,16674.750000,16116.250000,16499.600000,16850.558633,1.0,17395.250000,16946.050000,17058.350000,-657.700000,16946.050000
25%,2019-03-13 12:48:45,26262.900000,26316.012500,26208.687500,26265.162500,26270.806495,1.0,24496.100000,24103.600000,24137.600000,-130.400000,26744.437500
50%,2020-11-11 10:22:30,31400.275000,31458.325000,31334.000000,31393.550000,31380.240673,1.0,30627.550000,30256.900000,30270.650000,-70.300000,31899.350000
75%,2022-07-12 09:26:15,38570.187500,38666.100000,38468.412500,38571.450000,38576.653683,1.0,38480.900000,37425.600000,37661.550000,273.675000,38330.575000
max,2024-03-07 15:15:00,48547.400000,48631.000000,48508.500000,48548.550000,48508.280192,1.0,48601.000000,48193.750000,48280.900000,3287.700000,48193.750000
std,NaN,7590.527125,7594.645637,7586.580701,7590.748730,7587.438620,0.0,7837.180496,7711.708479,7733.194552,373.785513,7571.604945


In [19]:
data45B.index = data45B["Date_Time"]


In [20]:
data45B["PROFIT"].resample("A").sum()


Date_Time
2017-12-31     1881.70
2018-12-31     6070.75
2019-12-31     9121.95
2020-12-31    12864.30
2021-12-31    11239.45
2022-12-31    10675.90
2023-12-31     7525.30
2024-12-31     4852.70
Freq: A-DEC, Name: PROFIT, dtype: float64

In [21]:
#data45B.to_excel("5EMA_BUY_BN.xlsx")

Combining Long and Short and applying together

In [22]:
test_data45 = data45[["Date_Time", "POSITION", "PROFIT"]]
test_data45B = data45B[["Date_Time", "POSITION", "PROFIT"]]


In [23]:
data45Bi = 0
data45i = 0
position = None
final_data = {
    "Entry_Date_Time" : [],
    "Position" : [],
    "Exit_Date_Time": [],
    "Profit" : []
}
def calculate_profit(i45, i45B):
     global position, final_data, test_data45, test_data45B

     if position == None:

        if test_data45B.iloc[i45B, 1] == "ENTRY":
            position = "LONG"
            final_data["Entry_Date_Time"].append(test_data45B.iloc[i45B, 0])
            final_data["Position"].append("Long")

        elif test_data45.iloc[i45, 1] == "ENTRY":
            position = "SHORT"
            final_data["Entry_Date_Time"].append(test_data45.iloc[i45, 0])
            final_data["Position"].append("Short")

     elif position == "LONG":
            if test_data45B.iloc[i45B, 1] == "EXIT":
                final_data["Exit_Date_Time"].append(test_data45B.iloc[i45B, 0])
                final_data["Profit"].append(test_data45B.iloc[i45B, 2])
                position = None

     elif position == "SHORT":
            if test_data45.iloc[i45, 1] == "EXIT":
                final_data["Exit_Date_Time"].append(test_data45.iloc[i45, 0])
                final_data["Profit"].append(test_data45.iloc[i45, 2])
                position = None
                
while data45Bi < len(test_data45B) and data45i < len(test_data45):
    if data45B.iloc[data45Bi, 0] == data45.iloc[data45i, 0]:
        # Handle the case where timestamps are the same in both datasets
        # You can implement one of the approaches mentioned above
        # For example, you might process both entries or prioritize one over the other
        pass
    elif data45B.iloc[data45Bi, 0] > data45.iloc[data45i, 0]:
        calculate_profit(data45i, data45Bi)
        data45i += 1
    else:
        calculate_profit(data45i, data45Bi)
        data45Bi += 1


KeyboardInterrupt: 

In [ ]:
final_data["Exit_Date_Time"].append(None)
final_data["Profit"].append(None)

In [ ]:
final_data = pd.DataFrame(final_data)


ValueError: All arrays must be of the same length